In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
class SimpleLinearModel(nn.Module):
    def __init__(self):
        super(SimpleLinearModel, self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=20, bias=False)
        self.fc2 = nn.Linear(in_features=20, out_features=15, bias=False)
        self.fc3 = nn.Linear(in_features=15, out_features=1, bias=False)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
model = SimpleLinearModel()

In [4]:
num_params = sum(p.numel() for p in model.parameters())
num_params

515

In [5]:
memory_before_moving  = torch.cuda.memory_allocated()
print("==== memory before moving model to gpu ==", memory_before_moving)

==== memory before moving model to gpu == 0


In [6]:
model.cuda()
memory_before_forward = torch.cuda.memory_allocated() - memory_before_moving
print("==== memory before forward pass ==", memory_before_forward)

==== memory before forward pass == 3072


In [7]:
print(torch.cuda.memory_allocated())

3072


In [8]:
param_memory = sum(p.nelement() * p.element_size() for p in model.parameters())
print(f'Total parameter memory: {param_memory} bytes')

Total parameter memory: 2060 bytes


In [9]:
input = torch.randn(1, 10).cuda()
print(torch.cuda.memory_allocated())

3584


In [123]:
output = model(input)
memory_after = torch.cuda.memory_allocated()
memory_usage = memory_after - memory_before_forward
print(f'Memory used during forward pass: {memory_usage} bytes')

Memory used during forward pass: 2048 bytes


In [124]:
print(torch.cuda.memory_allocated())
print(memory_before_forward, memory_before_forward)

5120
3072 3072


In [127]:
torch.cuda.empty_cache()
del model
import gc
gc.collect()

NameError: name 'model' is not defined

In [128]:
print(torch.cuda.memory_allocated())

5120


In [10]:
output = model.fc1(input)
print('Memory after fc1:', torch.cuda.memory_allocated())

output = F.relu(output)
print('Memory after relu1:', torch.cuda.memory_allocated())

output = model.fc2(output)
print('Memory after fc2:', torch.cuda.memory_allocated())

output = F.relu(output)
print('Memory after relu2:', torch.cuda.memory_allocated())

output = model.fc3(output)
print('Memory after fc3:', torch.cuda.memory_allocated())


Memory after fc1: 4096
Memory after relu1: 4096
Memory after fc2: 4608
Memory after relu2: 4608
Memory after fc3: 5120
